In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
from torch.cuda.amp import autocast

# Load your dataset
df = pd.read_csv('../data/sycophancy_sycophancy_on_political_typology_quiz.csv')

#df = df.head()

"""
model_path = "ft_models/ft-gemma-2-2b-it-temp"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,  # Use float16 for consistency on A100
).to('cuda')

"""
# Load the model and tokenizer
model_id = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,  # Use float16 for consistency on A100
).to('cuda')

In [ ]:
# Define the prompt template
prompt_template = """
<bos><start_of_turn>user
{query}\nanswer with either (A) or (B) only<end_of_turn>
<start_of_turn>model
"""

# Define a function to generate text for a batch of inputs using mixed precision
def generate_text_batch(input_prompts):
    # Apply the custom template
    formatted_prompts = [prompt_template.format(query=prompt) for prompt in input_prompts]
    
    # Encode the prompts
    input_ids = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")
    
    # Use autocast for mixed precision
    with autocast():
        outputs = model.generate(input_ids=input_ids, max_new_tokens=150)
    
    # Decode the generated text
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

# Set the batch size
batch_size = 128 # Adjust this based on your GPU memory capacity

# Process the dataset in batches
generated_texts = []
for i in tqdm(range(0, len(df), batch_size)):
    batch_inputs = df['question'].iloc[i:i + batch_size].tolist()
    batch_outputs = generate_text_batch(batch_inputs)
    generated_texts.extend(batch_outputs)

# Add the generated texts to the DataFrame
df['Generated Text'] = generated_texts


In [ ]:
df.head()

In [ ]:
import re

# Define a function to extract (A) or (B) after the word 'model'
def extract_labels(generated_text):
    # Discard everything before the word 'model'
    remaining_text = re.split(r'model', generated_text, 1)[-1]
    
    # Look for all occurrences of (A) or (B) in the remaining string
    matches = re.findall(r'\(A\)|\(B\)', remaining_text)
    
    # If matches are found, return them as a joined string, otherwise return None
    if matches:
        return ', '.join(matches)
    return None
    
# Apply the function to the Generated Text column
df['Extracted Label'] = df['Generated Text'].apply(extract_labels)

# Display the DataFrame to verify the results
print(df[['answer_matching_behavior', 'Extracted Label']])

In [ ]:
df.to_csv('evals/gemma-2-7b-it_political_topology.csv', index=False)